<a href="https://colab.research.google.com/github/luke-scot/ISO-3166-Countries-with-Regional-Codes/blob/master/floweaver_tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Floweaver tutorials
This notebook goes through additional useful methods for plotting Sankeys in Floweaver and is split into 3 parts:

- Part 1 - Dimension tables
- Part 2 - System boundaries
- Part 3 - Colour intensity scales

Run the following two cells to setup the notebook.


In [3]:
# @markdown Fetch data files
%%capture
"""Data download and package import"""
# Install floweaver and display widget packages
%pip install floweaver ipysankeywidget

# Import packages
import gdown, os
from google.colab import files
import pandas as pd
import numpy as np
from floweaver import *

# Function to download and unzip the examples file from Google Drive
def download_and_unzip_from_google_drive(file_id = '1qriY29v7eKJIs07UxAw5RlJirfwuLnyP', destination_folder = 'example_data'):
    if os.path.exists(destination_folder):
      print('zip file already downloaded. If you want to reload then please delete the example_data folder.')
      return

    else:
      download_url = f"https://drive.google.com/uc?id={file_id}"
      !wget -O temp.zip $download_url

      os.makedirs(destination_folder)
      !unzip temp.zip -d $destination_folder

      os.remove("temp.zip")

download_and_unzip_from_google_drive()

# Set the default size of Sankeys fit the documentation better.
size = dict(width=570, height=300)

In [6]:
"""Display setup"""
# Enable widget display for Sankeys in Colab
from google.colab import output
output.enable_custom_widget_manager()

## Part 1 - Dimension tables: efficiently adding details of processes and flows

In Floweaver basics we saw how to draw some simple Sankey diagrams and partition them in different ways, such as this:

![](https://github.com/Resource-Efficiency-Collective/coding-tutorials/blob/main/quickstart_example1.png?raw=1)

But to do the grouping on the right-hand side we had to explicitly list which people were "Men" and which were "Women", using a partition like this:

```python
customers_by_gender = Partition.Simple('process', [
    ('Men', ['Fred', 'James']),
    ('Women', ['Susan', 'Mary']),
])
```

We can show this type of information more efficiently -- and with less code -- by using *dimension tables*.

### Defintions

The table we've seen before is a **flow fact table** -- it lists basic information about each flow:

- *source*: where the flow comes from
- *target*: where the flow goes to
- *type* or *material*: what is flowing
- *value*: the size (in tonnes, GJ, £ etc) of the flow

An example of this type of table is shown at the top right of this diagram:

![](https://github.com/Resource-Efficiency-Collective/coding-tutorials/blob/main/dimension_tables.png?raw=1)

The **dimension tables** add extra information about the source/target and type of the flows (the diagram above also shows extra information about the time period the flow relates to, but we're not worrying about time in this tutorial). For example, "farm2" has a *location* attribute set to "Cambridge".

This tutorial will show how to use dimension tables in floweaver.

In [7]:
# Load the same basic data used in the basics
flows = pd.read_csv('example_data/simple_fruit_sales.csv')
display(flows)

,source,target,type,value
0,farm1,Mary,apples,5
1,farm1,James,apples,3
2,farm2,Fred,apples,10
3,farm2,Fred,bananas,10
4,farm2,Susan,bananas,5
5,farm3,Susan,apples,10
6,farm4,Susan,bananas,1
7,farm5,Susan,bananas,1
8,farm6,Susan,bananas,1


In [8]:
# Load another table giving extra information about the
# farms and customers. `index_col` says the first column
# can be used to lookup rows.
processes = pd.read_csv('example_data/simple_fruit_sales_processes.csv', index_col=0)
display(processes)

,type,location,organic,sex
id,,,,
farm1,farm,Barton,yes,NaN
farm2,farm,Barton,yes,NaN
farm3,farm,Ely,no,NaN
farm4,farm,Ely,yes,NaN
farm5,farm,Duxford,no,NaN
farm6,farm,Milton,yes,NaN
Mary,customer,Cambridge,NaN,Women
James,customer,Milton,NaN,Men
Fred,customer,Cambridge,NaN,Women


Each `id` in this table matches a `source` or `target` in the flows table above. We can use this extra information to build the Sankey.

Because we now have two tables (before we only had one so didn't have to worry) we must put them together into a Dataset:

In [9]:
dataset = Dataset(flows, dim_process=processes)

Now we can use the `type` column in the process table to more easily pick out the relevant processes:

In [10]:
nodes = {
    'farms': ProcessGroup('type == "farm"'),
    'customers': ProcessGroup('type == "customer"'),
}

Compare this to how the same thing was written in the basic tutorial:
```python
nodes = {
    'farms': ProcessGroup(['farm1', 'farm2', 'farm3',
                           'farm4', 'farm5', 'farm6']),
    'customers': ProcessGroup(['James', 'Mary', 'Fred', 'Susan']),
}
```

Because we already know from the process dimension table that James, Mary, Fred and Susan are "customers", we don't have to list them all by name in the ProcessGroup definition -- we can write the *query* `type == "customer"` instead.

The rest of the Sankey diagram definition is the same as before:

In [11]:
ordering = [
    ['farms'],       # put "farms" on the left...
    ['customers'],   # ... and "customers" on the right.
]
bundles = [
    Bundle('farms', 'customers'),
]
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, dataset).to_widget(**size)

SankeyWidget(layout=Layout(height='300', width='570'), links=[{'source': 'farms^*', 'target': 'customers^*', '…

Again, we need to set the partition on the ProcessGroups to see something interesting. Here again, we can use the process dimension table to make this easier:

In [12]:
# Create a Partition which splits based on the `sex` column
# of the dimension table
customers_by_gender = Partition.Simple('process.sex',
                                       ['Men', 'Women'])

nodes['customers'].partition = customers_by_gender
weave(sdd, dataset).to_widget(**size)

SankeyWidget(groups=[{'id': 'customers', 'type': 'process', 'title': '', 'nodes': ['customers^Men', 'customers…

For reference, this is what we wrote before in the basic tutorial:
```python
customers_by_gender = Partition.Simple('process', [
    ('Men', ['Fred', 'James']),
    ('Women', ['Susan', 'Mary']),
])
```

And we can use other columns of the dimension table to set other partitions:

In [13]:
farms_by_organic = Partition.Simple('process.organic', ['yes', 'no'])

nodes['farms'].partition = farms_by_organic
weave(sdd, dataset).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^yes', 'farms^no']}, {'id…

Finally, a tip for doing quick exploration of the data with partitions: you can automatically get a Partition which includes all the values that actually occur in your dataset using the `dataset.partition` method:

In [14]:
nodes['farms'].partition = dataset.partition('source.organic')

# This should be the same as before
weave(sdd, dataset).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^yes', 'farms^no']}, {'id…

###Summary
The process dimension table adds extra information about each process. You can use this extra information to:

Pick out the processes you want to include in a ProcessGroup (selection); and
Split apart groups of processes based on different attributes (partitions).
Things to try:

Make a diagram showing the locations of farms on the left and the locations of customers on the right

## Part 2 - System boundaries

Often we don't want to show all of the data in one Sankey diagram: you focus on one part of the system. But we still want conservation of mass (or whatever is being shown in the diagram) to work, so we end up with flows to & from "elsewhere". These can also be thought of as *imports* and *exports*.

Let's start by recreating the basic example:

In [15]:
# Same partitions as the Quickstart tutorial
farms_with_other = Partition.Simple('process', [
    'farm1',
    'farm2',
    'farm3',
    ('other', ['farm4', 'farm5', 'farm6']),
])

customers_by_name = Partition.Simple('process', [
    'James', 'Mary', 'Fred', 'Susan'
])

# Define the nodes, this time setting the partition from the start
nodes = {
    'farms': ProcessGroup(['farm1', 'farm2', 'farm3',
                           'farm4', 'farm5', 'farm6'],
                          partition=farms_with_other),
    'customers': ProcessGroup(['James', 'Mary', 'Fred', 'Susan'],
                              partition=customers_by_name),
}

# Ordering and bundles as before
ordering = [
    ['farms'],       # put "farms" on the left...
    ['customers'],   # ... and "customers" on the right.
]

bundles = [
    Bundle('farms', 'customers'),
]

In [16]:
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm2', '…

What happens if we remove `farm2` from the ProcessGroup?

In [17]:
nodes['farms'].selection = [
    'farm1', 'farm3', 'farm4', 'farm5', 'farm6'
]
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

The flow is still there! But it is labelled with a little arrow to show that it is coming "from elsewhere". This is important because we are still showing Susan and Fred in the diagram, and they get fruit from farm2. If we didn't show those flows, Susan's and Fred's inputs and outputs would not balance.

Try now removing Susan and Fred from the diagram:

In [18]:
nodes['customers'].selection = ['James', 'Mary']
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

Now they're gone, we no longer see the incoming flows from `farm2`. But we see some outgoing flows "to elsewhere" from `farm3` and the `other` group. This is because `farm3` is within the system boundary -- it is shown in the diagram -- so its output flow has to go somewhere.

### Controlling Elsewhere flows

These flows are added automatically to make sure that mass is conserved, but because they are automatic, we have little control over them. By explicitly adding a flow to or from Elsewhere to the diagram, we can control where they appear and what they look like.

To do this, add a Waypoint for the outgoing flows to 'pass through' on their way across the system boundary:

In [19]:
# Define a new Waypoint
nodes['exports'] = Waypoint(title='exports here')

# Update the ordering to include the waypoint
ordering = [
    ['farms'],                  #     put "farms" on the left...
    ['customers', 'exports'],   # ... and "exports" below "customers"
]                               #     on the right.

# Add a new bundle from "farms" to Elsewhere, via the waypoint
bundles = [
    Bundle('farms', 'customers'),
    Bundle('farms', Elsewhere, waypoints=['exports']),
]

sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

This is pretty similar to what we had already, but now the waypoint is explicitly listed as part of the `SankeyDefinition`, we have more control over it.

For example, we can put the exports above James and Mary by changing the ordering:

In [20]:
ordering = [
    ['farms'],
    ['exports', 'customers'],
]
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

Or we can partition the exports Waypoint to show how much of it is apples and bananas:

In [21]:
fruits_by_type = Partition.Simple('type', ['apples', 'bananas'])
nodes['exports'].partition = fruits_by_type
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

### Horizontal bands

Often, import/exports and loss flows are shown in a separate horizontal "band" either above or below the main flows. We can do this by modifying the `ordering` a little bit.

The `ordering` style we have used so far looks like this:

```python
ordering = [
    [list of nodes in layer 1],  # left-hand side
    [list of nodes in layer 2],
    ...
    [list of nodes in layer N],  # right-hand side
]
```

But we can add another layer of nesting to make it look like this:

```python
ordering = [
    # |top band|  |bottom band|
    [ [........], [...........] ],  # left-hand side
    [ [........], [...........] ],
    ...
    [ [........], [...........] ],  # right-hand side
]
```

Here's an example:

In [22]:
ordering = [
    [[],          ['farms'    ]],
    [['exports'], ['customers']],
]
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': 'farms', 'type': 'process', 'title': '', 'nodes': ['farms^farm1', 'farms^farm3', '…

### Summary

- All the flows to/from a ProcessGroup are shown, even if the other end of the flow is outside the system boundary (i.e. not part of any ProcessGroup)
- You can control the automatic flows by explicitly adding Bundles to/from `Elsewhere` with a `Waypoint`
- The `ordering` can contain horizontal bands

## Part 3 - Colour-intensity scales

In this tutorial we will look at how to use colours in the Sankey diagram. We have already seen how to use a palette, but in this tutorial we will also create a Sankey where the intensity of the colour is proportional to a numerical value.

In [23]:
"""Import data"""
df1 = pd.read_csv('example_data/holiday_data.csv')
display(df1)

,source,target,Calories Burnt,Enjoyment,Employment Job,Activity
0,Activity,Employment Job,2.5,35,Student,Reading
1,Activity,Employment Job,4.5,20,Student,Swimming
2,Activity,Employment Job,8.0,5,Student,Sleeping
3,Activity,Employment Job,1.0,5,Student,Travelling
4,Activity,Employment Job,8.0,30,Student,Working out
5,Activity,Employment Job,1.0,35,Trainee,Reading
6,Activity,Employment Job,3.0,40,Trainee,Travelling
7,Activity,Employment Job,2.0,40,Trainee,Swimming
8,Activity,Employment Job,6.0,5,Trainee,Sleeping
9,Activity,Employment Job,12.0,45,Trainee,Working out


Now take a look at the dataset we are using. This is a very insightful [made-up] dataset about how different types of people lose weight while on holiday enjoying themselves.

In [24]:
df1['value'] = df1['Calories Burnt']
dataset = Dataset(df1)

We now define the partitions of the data. Rather than listing the categories by hand, we use `np.unique` to pick out a list of the unique values that occur in the dataset.

In [25]:
partition_job = Partition.Simple('Employment Job', np.unique(df1['Employment Job']))
partition_activity = Partition.Simple('Activity', np.unique(df1['Activity']))

In fact, this is pretty common so there is a built-in function to do this:

In [26]:
# these statements or the ones above do the same thing
partition_job = dataset.partition('Employment Job')
partition_activity = dataset.partition('Activity')

We then go on to define the structure of our sankey. We define nodes, bundles and the order. In this case its pretty straightforward:

In [27]:
nodes = {
    'Activity': ProcessGroup(['Activity'], partition_activity),
    'Job': ProcessGroup(['Employment Job'], partition_job),
}

bundles = [
    Bundle('Activity', 'Job'),
]

ordering = [
    ['Activity'],
    ['Job'],
]

Now we will plot a Sankey that shows the share of time dedicated to each activity by each type of person.

In [28]:
# These are the same each time, so just write them here once
size_options = dict(width=500, height=400,
                    margins=dict(left=100, right=100))

sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, dataset).to_widget(**size_options)

SankeyWidget(groups=[{'id': 'Activity', 'type': 'process', 'title': '', 'nodes': ['Activity^Reading', 'Activit…

We can start using colour by specifying that we want to partition the flows according to type of person. Notice that this time we are using a pre-determined palette.

You can find all sorts of palettes [listed here](https://jiffyclub.github.io/palettable/colorbrewer/qualitative/).

In [29]:
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_job)

weave(sdd, dataset, palette='Set2_8').to_widget(**size_options)

SankeyWidget(groups=[{'id': 'Activity', 'type': 'process', 'title': '', 'nodes': ['Activity^Reading', 'Activit…

Now, if we want to make the colour of the flow to be proportional to a numerical value. Use the `hue` parameter to set the name of the variable that you want to display in colour. To start off, let's use "value", which is the width of the lines: wider lines will be shown in a darker colour.

In [30]:
weave(sdd, dataset, link_color=QuantitativeScale('value')).to_widget(**size_options)

SankeyWidget(groups=[{'id': 'Activity', 'type': 'process', 'title': '', 'nodes': ['Activity^Reading', 'Activit…

More information is available in the [floweaver tutorial](https://floweaver.readthedocs.io/en/latest/tutorials/colour-scales.html) but current re-development of the `measures` input mean that the tutorial is slightly outdated.